In [1]:
import tensorflow as tf
import numpy as np
import sys
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt



In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train_set = mnist.train.images*2 - 1
train_labels = mnist.train.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
mnist.train.labels[0].shape

(10,)

In [ ]:
def generator(z, Y, reuse=None):
	with tf.variable_scope('gen',reuse=reuse):
        
		ip = tf.concat([z, Y], 1)
		hidden1 = tf.layers.dense(inputs=ip,units=128)

		alpha = 0.01
		hidden1 = tf.maximum(alpha*hidden1, hidden1)

		hidden2 = tf.layers.dense(inputs=hidden1,units=128)

		hidden2 = tf.maximum(alpha*hidden2,hidden2)

		output = tf.layers.dense(hidden2,units=784,activation=tf.nn.tanh)

		return output

In [ ]:
def discriminator(X, Y, reuse=None):
	with tf.variable_scope('dis',reuse=reuse):
        
		ip = tf.concat([X, Y], 1)
		hidden1 = tf.layers.dense(inputs=ip,units=128)

		alpha = 0.01
		hidden1 = tf.maximum(alpha*hidden1, hidden1)

		hidden2 = tf.layers.dense(inputs=hidden1,units=128)

		hidden2 = tf.maximum(alpha*hidden2,hidden2)

		logits = tf.layers.dense(hidden2, units=1)
		output = tf.sigmoid(logits)

		return output,logits

In [ ]:
real_images = tf.placeholder(tf.float32,shape=[None,784])
y = tf.placeholder(tf.float32, shape=(None, 10))
z = tf.placeholder(tf.float32,shape=[None,100])
onehot = np.eye(10)

G = generator(z, y)

D_output_real , D_logits_real = discriminator(real_images, y)

D_output_fake , D_logits_fake = discriminator(G, y, reuse=True)

In [ ]:
#LOSSES
def loss_func(logits_in, labels_in):
	return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
		logits=logits_in,labels=labels_in))

D_real_loss = loss_func(D_logits_real, tf.ones_like(D_logits_real)*0.9) #SMOOTHING
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_fake))

D_loss = D_real_loss + D_fake_loss

G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

learning_rate = 0.001

tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss,var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss,var_list=g_vars)

In [ ]:
batch_size = 100
epochs = 100
init = tf.global_variables_initializer()
samples = []

In [ ]:
with tf.Session() as sess:
	sess.run(init)

	for epoch in range(epochs):

		for i in range(len(train_set) // batch_size):

			#update discriminator
			x_ = train_set[i * batch_size:(i + 1) * batch_size]
			y_ = train_labels[i * batch_size:(i + 1) * batch_size]

			batch_z = np.random.uniform(-1,1,size=(batch_size,100))

			_, D_loss_curr = sess.run([D_trainer, D_loss] ,feed_dict={real_images:x_, y:y_ ,z:batch_z})
            
            #update generator
			y_ = np.random.randint(0, 9, (batch_size, 1))
			y_ = onehot[y_.astype(np.int32)].squeeze()
			_, G_loss_curr = sess.run([G_trainer, G_loss] ,feed_dict={z:batch_z, y:y_})

		print("ON EPOCH {}: D_loss={}, G_loss={}".format(epoch,D_loss_curr,G_loss_curr))

		sample_z = np.random.uniform(-1,1,size=(1,100))
		sample_y = np.random.randint(0, 9, (1, 1))
		sample_y = onehot[sample_y.astype(np.int32)].squeeze()
		sample_y = sample_y.reshape(1,10)
		gen_sample = sess.run(generator(z,y,reuse=True),feed_dict={z:sample_z, y:sample_y})

		samples.append(gen_sample)	
        
	saver = tf.train.Saver(var_list=g_vars)
	saver.save(sess, './models/30e_label.ckpt')

In [ ]:
%matplotlib inline
plt.imshow(samples[19].reshape(28,28))

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver(var_list=g_vars)
    saver.restore(sess, './models/30e_label.ckpt')
    res_samp = []
    
    for i in range(10):
        gen_z = np.random.uniform(-1,1,size=(1,100))
        gen_y = np.array([[8]])
        gen_y=onehot[gen_y.astype(np.int32)].squeeze().reshape(1,10)

        res_samp.append(sess.run(generator(z, y, reuse=True), feed_dict={z:gen_z, y:gen_y}))

In [ ]:
plt.imshow(res_samp[6].reshape(28,28), cmap='gist_gray')